In [1]:
%%capture

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from datetime import datetime
date_du_jour = datetime.today().strftime("%d_%m_%Y")
from pandas import DataFrame, ExcelWriter
import openpyxl

from comcare import driver
from caregiver import id_parent_served, parents, sdata, id_parent_completed 
from handler import name_handler

In [2]:
%%capture
id_parent_served.drop_duplicates('id_patient',inplace=True)
id_parent_completed.drop_duplicates('id_patient',inplace=True)
parents.drop_duplicates('code',inplace=True)
sdata.drop_duplicates('case_id',inplace=True)

In [3]:
id_parent_served.id_patient.count()
id_parent_completed.id_patient.count()
parents.code.count()
sdata.case_id.count()

218

218

662

82277

In [4]:
lc_idparentserved = sdata[sdata.id_patient.isin(id_parent_served.id_patient)]
lc_ips = lc_idparentserved[['id_patient','code']]
lc_idparentserved.code.count()
lc_ips.code.count()

218

218

In [5]:
code_in_parents = lc_ips[lc_ips.code.isin(parents.code)]
code_notIn_parents = lc_ips[~lc_ips.code.isin(parents.code)]

code_in_parents.code.count()
code_notIn_parents.code.count()

218

0

# the completed

In [6]:
id_parent_completed[id_parent_completed.id_patient.isin(code_in_parents.id_patient)].id_patient.count()
id_parent_completed[~id_parent_completed.id_patient.isin(code_in_parents.id_patient)].id_patient.count()

importantMissing = id_parent_completed[~id_parent_completed.id_patient.isin(code_in_parents.id_patient)]
importantMissing = sdata[sdata.id_patient.isin(importantMissing.id_patient)]

218

0


# COnclusion


- fosref must add "Code_notIn_parents" for the caregiver
- code_in_parents are Datim whorty

## not in parents

In [7]:
missing_FAP = sdata[sdata.id_patient.isin(code_notIn_parents.id_patient)]
missing_FAP.id_patient.count()

0

# in parents - datim

In [28]:
parents

,code,non_paran_responsab,Gender,commune,age_paran,age_ovc,age_status
0,PAP/DRMS/032129105,Rachelle Metellus,Female,kenscoff,32,18+,normal
1,PAP/DRMS/031160607,Cafina Telcy,Female,tabarre,28,18+,normal
2,PAP/DRMS/030158015,Veus Chalina,Female,tabarre,41,18+,normal
3,PAP/DRMS/122140802,Dieula Joseph Rosmé,Female,kenscoff,43,18+,normal
4,PAP/DRMS/097068702,hhhhhb,Female,tabarre,39,18+,normal
...,...,...,...,...,...,...,...
658,PAP/DRMS/178403409,jean charles,Female,port-au-prince,30,18+,normal
659,PAP/DRMS/178396410,leon,Female,port-au-prince,30,18+,normal
660,PAP/DRMS/093427611,nougaisse,Female,port-au-prince,30,18+,normal
661,PAP/DRMS/236418904,saint jean,Male,port-au-prince,30,18+,normal


In [52]:
found_FAP = sdata[sdata.id_patient.isin(code_in_parents.id_patient)]
found_FAP.id_patient.count()

cip = code_in_parents[['code']]
parents_reportable = cip.merge(parents,how='left',on='code')
parents_reportable.code.count()

218

218

In [58]:
fap = found_FAP[['code','commune']]

In [ ]:
fap.rename(
    columns={"commune":"commune_on_dsd"},
    inplace=True
)

In [63]:
parents_reportable  = fap.merge(parents_reportable,on="code",how='left')

In [65]:
parents_reportable.fillna("---",inplace=True)

In [72]:
parents_reportable

,code,commune_on_dsd,non_paran_responsab,Gender,commune,age_paran,age_ovc,age_status
0,PAP/DRMS/230425801,tabarre,Jeannot,Male,tabarre,30,18+,normal
1,PAP/DRMS/094281401,petion_ville,Pierre Saint,Male,petion_ville,30,18+,normal
2,PAP/DRMS/039370801,carrefour,Juissance,Male,carrefour,30,18+,normal
3,GON/DRMS/058245556,dessalines,Vixama,Male,dessalines,30,18+,normal
4,PAP/DRMS/046201301,desdunes,Dorval,Male,desdunes,30,18+,normal
...,...,...,...,...,...,...,...,...
213,GON/DRMS/062461915,verrettes,Jeantilus,Male,verrettes,30,18+,normal
214,GON/DRMS/062461918,verrettes,Filius,Male,verrettes,30,18+,normal
215,GON/DRMS/062093606,verrettes,Albert Marie Maude,Female,verrettes,40,18+,normal
216,GON/DRMS/065143528,desdunes,Francois Diamil,Male,desdunes,-1,18+,aCorrige


In [73]:



# assume commune_on_dsd is compplete

parents_reportable['commune'] = parents_reportable['commune_on_dsd'].where(
    parents_reportable['commune'].isin(["---"]),
    other= parents['commune']
)

In [74]:
parents_reportable.shape

(218, 8)

In [75]:
parents_reportable.Gender.value_counts()

Male      139
Female     79
Name: Gender, dtype: int64

In [76]:
parents_reportable.age_ovc.value_counts()

18+    218
Name: age_ovc, dtype: int64

In [77]:
parents_reportable.commune.value_counts().sum()

218

In [78]:
parents_reportable.pivot_table(index="commune",values="code",columns=["Gender","age_ovc"],aggfunc='count',fill_value=0)

Gender,Female,Male
age_ovc,18+,18+
commune,,
carrefour,1,0
delmas,1,3
desdunes,7,5
dessalines,9,7
grande-saline,2,3
kenscoff,39,81
la_chapelle,0,3
petion_ville,0,2


In [79]:
datim = parents_reportable.pivot_table(index="commune",values="code",columns=["Gender","age_ovc"],aggfunc='count',fill_value=0)


In [80]:
datim[('Female','<1')] = 0
datim[('Female','1-4')] = 0
datim[('Female','5-9')] = 0
datim[('Female','10-14')] = 0
datim[('Female','15-17')] = 0
datim[('Male','<1')] = 0
datim[('Male','1-4')] = 0
datim[('Male','5-9')] = 0
datim[('Male','10-14')] = 0
datim[('Male','15-17')] = 0

columns_orientation =[
    ('Female','<1'),
    ('Female','1-4'),
    ('Female','5-9'),
    ('Female','10-14'),
    ('Female','15-17'),   
    ('Female','18+'),  
    ('Male','<1'),
    ('Male','1-4'),
    ('Male','5-9'),
    ('Male','10-14'),
    ('Male','15-17'),   
    ('Male','18+'),   
]

datim = datim.reindex(columns= columns_orientation)
datim

Gender                       Female                         Male          \
age_ovc                          <1 1-4 5-9 10-14 15-17 18+   <1 1-4 5-9   
commune                                                                    
carrefour                         0   0   0     0     0   1    0   0   0   
delmas                            0   0   0     0     0   1    0   0   0   
desdunes                          0   0   0     0     0   7    0   0   0   
dessalines                        0   0   0     0     0   9    0   0   0   
grande-saline                     0   0   0     0     0   2    0   0   0   
kenscoff                          0   0   0     0     0  39    0   0   0   
la_chapelle                       0   0   0     0     0   0    0   0   0   
petion_ville                      0   0   0     0     0   0    0   0   0   
petite_rivire_de_lartibonite      0   0   0     0     0   1    0   0   0   
port-au-prince                    0   0   0     0     0   1    0   0   0   
saint-marc                        0   0   0     0     0   0    0   0   0   
tabarre                           0   0   0     0     0   9    0   0   0   
verrettes                         0   0   0     0     0   9    0   0   0   

Gender                                        
age_ovc                      10-14 15-17 18+  
commune                                       
carrefour                        0     0   0  
delmas                           0     0   3  
desdunes                         0     0   5  
dessalines                       0     0   7  
grande-saline                    0     0   3  
kenscoff                         0     0  81  
la_chapelle                      0     0   3  
petion_ville                     0     0   2  
petite_rivire_de_lartibonite     0     0   3  
port-au-prince                   0     0   0  
saint-marc                       0     0   3  
tabarre                          0     0  18  
verrettes                        0     0  11

# REPORT

In [81]:
#datim.to_excel(f"./caregiver_results/Q2/datim_parenting_caregiver{date_du_jour}.xlsx",na_rep="",index=True)
#found_FAP.to_excel(f"./caregiver_results/Q2/datim_caregiver_{date_du_jour}.xlsx",na_rep="",index=False)

#datim.to_excel(f"./caregiver_results/Q3/datim_parenting_caregiver{date_du_jour}.xlsx",na_rep="",index=True)
#found_FAP.to_excel(f"./caregiver_results/Q3/datim_caregiver_{date_du_jour}.xlsx",na_rep="",index=False)


#found_FAP.to_excel(f"./caregiver_results/datim_caregiver{date_du_jour}.xlsx",na_rep="",index=False)
#datim.to_excel(f"./caregiver_results/datim_parenting_caregiver{date_du_jour}.xlsx",na_rep="",index=True)

found_FAP.to_excel(f"./caregiver_results/datim_caregiver.xlsx",na_rep="",index=False)
datim.to_excel(f"./caregiver_results/datim_parenting_caregiver.xlsx",na_rep="",index=True)

## No konsaantman photo

In [ ]:
parents_reportable[parents_reportable['photo_konsantman'] =="---"].shape

In [ ]:
parents_reportable[parents_reportable['photo_konsantman'] =="---"]

# no Datim

missing_FAP_communes = []
for columns in missing_FAP.commune.unique():
    globals()[f"missing_FAP_{name_handler(columns)}"] = missing_FAP[missing_FAP.commune == columns]
    missing_FAP_communes.append(globals().get(f"missing_FAP_{name_handler(columns)}"))
    

importantMissing_communes = []
for columns in importantMissing.commune.unique():
    globals()[f"importantMissing_{name_handler(columns)}"] = importantMissing[importantMissing.commune == columns]
    importantMissing_communes.append(globals().get(f"importantMissing_{name_handler(columns)}"))
    


mfap_communes = missing_FAP.commune.unique().tolist()
missing_FAP_dict = {}
for columns, communes in zip(missing_FAP.commune.unique(), mfap_communes):
    missing_FAP_dict[f"{name_handler(communes)}"] =  globals().get(f"missing_FAP_{name_handler(columns)}").code.count()
        
missing_FAP_df = DataFrame.from_dict(missing_FAP_dict, orient="index", columns=None)
missing_FAP_df.columns = ["nombre_agyw/caregiver_KAP_pour_Regularisation"]
missing_FAP_df



im_communes = importantMissing.commune.unique().tolist()
importantMissing_dict = {}
for columns, communes in zip(importantMissing.commune.unique(), im_communes):
    importantMissing_dict[f"{name_handler(communes)}"] =  globals().get(f"importantMissing_{name_handler(columns)}").code.count()
    
importantMissing_df = DataFrame.from_dict(importantMissing_dict, orient="index", columns=None)
importantMissing_df.columns = ["nombre_agyw/caregiver_KAP_pour_Regularisation"]
importantMissing_df

importantMissing.to_excel('./missing/caregiver_CP_missing.xlsx',index=False,na_rep="")

missfap = ExcelWriter(f"./missing/missing_FAP_{date_du_jour}.xlsx",engine="openpyxl")
missing_FAP_df.to_excel(missfap,sheet_name='Resume',index=True)
missing_FAP.to_excel(missfap,sheet_name="Total", na_rep="",index=False)
for columns in missing_FAP.commune.unique():
    globals().get(f"missing_FAP_{name_handler(columns)}").to_excel(missfap,sheet_name=f"{name_handler(columns)}",index=False,na_rep="NULL")
missfap.save()

IM = ExcelWriter(f"./missing/caregiver_CP_missing.xlsx",engine="openpyxl")
importantMissing_df.to_excel(IM,sheet_name='Resume',index=True)
importantMissing.to_excel(IM,sheet_name="Total", na_rep="",index=False)
for columns in importantMissing.commune.unique():
    globals().get(f"importantMissing_{name_handler(columns)}").to_excel(IM,sheet_name=f"{name_handler(columns)}",index=False,na_rep="NULL")
IM.save()

In [82]:
driver.quit()